## 2.3. 어텐션 이해하기

어텐션이란 텍스트를 처리하는 관점에서는 입력한 테긋트에서 어떤 단어가 서로 관련되는지 '주의를 기울여' 파악한다.

어텐션에서는 쿼리, 키, 값 이라는 중요한 개념이 등장한다.

### 2.3.1. 사람이 글을 읽는 방법과 어텐션

사람이 글을 읽을 때, 쉬운 글을 왼쪽에서 오른쪽으로 순차적으로 읽지만 어려운 글은 어떤 단어가 어떤 단어에 관련되어 있는지 파악하기 위해 여러 번 읽는다. 따라서 사람은 그 단어 만으로 해석하지 않고 주변 단어와의 관계를 파악해서 맥락을 이해한다.

따라서 자연스럽게 관련이 있는 단어를 찾고 그 맥락을 이해하기 위해서 단어와 단어 사이의 관계를 계산해서 그 값에 따라 관련이 깊은 단어와 그렇지 않은 단어를 구분할 수 있어야 한다.

### 2.3.2. 쿼리, 키, 값 이해하기

쿼리 키 값은 정보 검색 분야에서 가져온 용어다. 예문에서 "파리"는 쿼리가 되고, "파리"와 관련된 단어의 특징을 "키"로 정의하고 실제 단어를 "값"으로 정의한다.

**"단어 모두에게 같은 가중치를 준다"는 가설**은 단어와 단어 사이의 관계를 고려하지 않는다.

**"가까이 있는 단어가 관련이 있이 높다" 가설**은 단어와 단어 사이의 관계를 고려하지만, 문법이나 의미적인 관계를 고려하지 않는다.

**$Wq$, $Wk$** 
간접적인 관련성과 같은 단어의 관련도가 높게 계산되는 문제를 해결하기위해서 트랜스포머 아키텍처에서는 토큰 임베딩을 변환하는 가중치 행렬 $Wq$, $Wk$를 사용한다. 이 가중치 행렬을 사용하면 단어와 단어 사이의 관계를 고려할 수 있다.



In [3]:
# 예제 2.4 쿼리, 키, 값 벡터를 만드는 nn.Linear 층
import torch
import torch.nn as nn

input_text = "나는 최근 파리 여행을 다녀왔다"

input_text_list = input_text.split()
print("토큰화 결과: ", input_text_list)

# 토큰 -> 아이디 딕셔너리 생성
str2dix = {word: i for i, word in enumerate(input_text_list)}
print("토큰 -> 아이디 딕셔너리: ", str2dix)
# 아이디 -> 토큰 딕셔너리 생성
dix2str = {i: word for i, word in enumerate(input_text_list)}
print("아이디 -> 토큰 딕셔너리: ", dix2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2dix[word] for word in input_text_list]
print("토큰을 아이디로 변환: ", input_ids)

embedding_dim = 16 # 임베딩 차원
head_dim = 16 # 헤드 차원

embed_layer = nn.Embedding(len(str2dix), embedding_dim) # 임베딩 레이어 생성

input_embeddings = embed_layer(torch.tensor(input_ids)) # 토큰 아이디를 임베딩 벡터로 변환
input_embeddings = input_embeddings.unsqueeze(0) # 배치 차원 추가

# 쿼리, 키, 값을 게산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

# 변환
querys = weight_q(input_embeddings) # 쿼리
keys = weight_k(input_embeddings) # 키
values = weight_v(input_embeddings) # 값

토큰화 결과:  ['나는', '최근', '파리', '여행을', '다녀왔다']
토큰 -> 아이디 딕셔너리:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
아이디 -> 토큰 딕셔너리:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
토큰을 아이디로 변환:  [0, 1, 2, 3, 4]


In [5]:
# 예제 2.5 스케일 점곱 방식의 어텐션

from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):

    dim_k = querys.size(-1)

    # 쿼리와 키를 곱하고 임베딩 차원 수의 제곱근으로 나눈다 (스케일)
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    # 쿼리와 키를 곱해 계산한 스코어 값이 1이 되도록 소프트맥스를 취해 가중치를 구한다
    weights = F.softmax(scores, dim=-1) # 소프트맥스
    # 가중치와 값을 곱해 입력과 동일한 차원을 가진 출력을 만든다
    return weights @ values # 가중합


In [7]:
# 예제 2.6 어텐션 함수를 이용한 어텐션 계산

print("원본 입력 형태: ", input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print("어텐션 후 입력 형태: ", after_attention_embeddings.shape)

원본 입력 형태:  torch.Size([1, 5, 16])
어텐션 후 입력 형태:  torch.Size([1, 5, 16])


### 2.3.4. 멀티 헤드 어텐션

- 트랜스포머의 핵심 구성 요소 중 하나인 멀티 헤드 어텐션에 대해 알아보자.
- 멀티 헤드 어텐션은 어텐션을 병렬로 수행하여 어텐션의 품질을 높이는 방법이다.

```mermaid
graph TD
    q[Query]
    k[Key]
    v[Value]

    linear1[Linear]
    linear2[Linear]
    linear3[Linear]

    q --> linear1
    k --> linear2
    v --> linear3

    scale[Scale Dot Product * h]
    connect[Connect]
    final_linear[Final Linear]

    linear1 --> scale
    linear2 --> scale
    linear3 --> scale

    scale --> connect
    connect --> final_linear
```

In [12]:
# 예제 2.7 어텐션 연산을 수행하는 AttentionHead 클래스

class AttentionHead(nn.Module):
    def __init__(self, embedding_dim, head_dim, is_causal=False):
        super().__init__()
        self.weight_q = nn.Linear(embedding_dim, head_dim) # 쿼리 벡터 생성을 위한 선형층
        self.weight_k = nn.Linear(embedding_dim, head_dim) # 키 벡터 생성을 위한 선형층
        self.weight_v = nn.Linear(embedding_dim, head_dim) # 값 벡터 생성을 위한 선형층
        self.is_causal = is_causal

    def forward(self, querys, keys, values):
        outputs = compute_attention(
            self.weight_q(querys),
            self.weight_k(keys),
            self.weight_v(values),
            is_causal=self.is_causal
        )
        return outputs

attention_head = AttentionHead(embedding_dim, head_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)

In [13]:
# 멀티 헤드 어텐션 구현

import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, token_embed_dim, d_model, h_head, is_causal=False):
        super().__init__()
        self.n_head = h_head
        self.is_causal = is_causal
        self.weight_q = nn.Linear(token_embed_dim, d_model)
        self.weight_k = nn.Linear(token_embed_dim, d_model)
        self.weight_v = nn.Linear(token_embed_dim, d_model)
        self.concat_linear = nn.Linear(d_model, d_model)

    def forward(self, querys, keys, values):
        
        B, T, C = querys.size()
        querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # 1번째 선형 층 적용
        keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # 2번째 선형 층 적용
        values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # 3번째 선형 층 적용

        attention = compute_attention(querys, keys, values, self.is_causal) # h번의 스케일 점곱 어텐션 수행
        output = attention.transpose(1, 2).reshape(B, T, C) # 어텐션 결과를 연결
        output = self.concat_linear(output) # 마지막 선형층 적용
        return output
    
n_head = 4
mh_attention = MultiHeadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape

torch.Size([1, 5, 16])

## 2.4. 정규화와 피드 포워드 층

정규화란, 딥러닝 모델에서 입력이 일정한 분포를 갖도록 만들어 학습이 안정적이고 빠르게 이루어지도록 하는 방법이다. 과거에는 배치 정규화(Batch Normalization)이 주로 사용되었지만, 트랜스포머 아키텍처에서는 특징 차원에서 정규화를 수행하는 층 정규화(Layer Normalization)를 사용한다.
어텐션 연산이 입력 단어 사이의 관계를 계산해 토큰 임베딩을 조정하는 역할을 한다면 전체 입력문장을 이해하는 연산은 완전 연결 층(Fully Connected Layer)이다. 이 층은 입력과 출력의 차원이 같은 행렬 곱셈을 수행하고, 이후에 활성화 함수를 통과시켜 비선형성을 추가한다. 이러한 층을 피드 포워드 층(Feed Forward Layer)이라고 한다.

### 2.4.1. 층 정규화 이해하기

벡터 $x$를 정규화한 $norm_x$는 벡터 $x$에서 $x$의 평균을 빼고 $x$의 표준편차로 나눠 평균이 $0$이고 표준편차가 $1$인 분포로 만든다.
$$
\text{norm}_x = \frac{x - \mu}{\sigma}
$$

**배치 정규화(Batch Normalization)**

배치 정규화는 모델에 입력으로 들어가는 미니 배치 사이의 정규화를 수행한다. 자연어 처리에서는 배치 정규화를 사용하지 않는 이유는 자연어 처리에서 배치에 들어가는 입력의 길이가 다양하기 때문이다.

**층 정규화(Layer Normalization)**

층 정규화는 배치 정규화와 달리 각 토큰 임베딩의 평균과 표준편차를 구해 정규화를 수행한다. 이를 통해 입력의 분포를 안정화시킬 수 있다.
이려면 배치 정규화의 단점을 보완할 수 있다.

**층 정규화의 2가지 방식**

**사후 정규화(Post-LN)**
어텐션관 피드 포워드 층 이후에 층 정규화를 수행하는 방식이다.

**사전 정규화(Pre-LN)**
층 정규화를 수행한 후에 어텐션과 피드 포워드 층을 수행하는 방식이다.

In [10]:
# 2.9 층 정규화 (사전 정규화)

import torch.nn as nn

norm = nn.LayerNorm(embedding_dim)
norm_x = norm(after_attention_embeddings)
print("norm_x shape: ", norm_x.shape)

print(norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data) # 실제로 평균과 표준편차 확인

norm_x shape:  torch.Size([1, 5, 16])
tensor([[-1.8626e-09,  0.0000e+00,  0.0000e+00, -2.2352e-08, -2.9802e-08]]) tensor([[1.0327, 1.0326, 1.0326, 1.0328, 1.0327]])
